In [1]:
#Data loading
import pandas as pd
dataset_with_null = pd.read_excel("C:\\Users\\91628\\Desktop\\smlprojectcleaneddataset (1).xlsx")
dataset_with_null['days_to_trend'] = dataset_with_null['trending_Date']-dataset_with_null['published_date']
data_og = dataset_with_null.dropna()

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91628\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd
from nltk.corpus import stopwords
from collections import Counter
import re
import string

dataset = data_og
def preprocess_text(text):
    # Remove non-ASCII characters
    text = ''.join([char for char in text if char in string.printable])
    return text

def remove_stopwords(text, stop_words):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def process_dataframe(dataset, stop_words):
    # Apply preprocessing to the 'title' column
    dataset['title'] = dataset['title'].apply(preprocess_text)

    # Remove stopwords from the 'title' column
    dataset['title'] = dataset['title'].apply(lambda x: remove_stopwords(x, stop_words))

    return dataset

stop_words = set(stopwords.words('english')) | {'|','1','2','3','4','5','6','7','8','9','0','vs','le','la','de','none'}
dataset = process_dataframe(dataset, stop_words)

In [4]:
#get frequencies of each word
import pandas as pd
from collections import Counter
import re

data_test = dataset[:100]

def replace_with_word_frequencies(df, column_name):
    # Combine all titles into a single string
    all_titles = ' '.join(df[column_name].astype(str).tolist())

    # Tokenize the text into words
    words = re.findall(r'\b\w+\b', all_titles.lower())

    # Calculate word frequencies using Counter for the entire column
    word_frequencies = Counter(words)

    # Function to replace words with their total occurrences in the column
    def replace_with_frequency(title):
        title_words = re.findall(r'\b\w+\b', str(title).lower())
        frequencies = [word_frequencies[word] for word in title_words]
        return frequencies

    # Apply the function to the column and create a new column
    new_column_name = f"{column_name}_frequencies"
    df[new_column_name] = df[column_name].apply(replace_with_frequency)

    return df


# The function modifies the DataFrame in place, so there's no need for assignment
dataset = replace_with_word_frequencies(dataset, 'title')

In [5]:
import pandas as pd
from collections import Counter
import re

data_test = dataset[:100]

def replace_with_word_frequencies(df, column_name):
    # Combine all titles into a single string
    all_titles = ' '.join(df[column_name].astype(str).tolist())

    # Tokenize the text into words
    words = re.findall(r'\b\w+\b', all_titles.lower())

    # Calculate word frequencies using Counter
    word_frequencies = Counter(words)

    # Function to replace words with their frequencies
    def replace_with_frequency(title):
        title_words = re.findall(r'\b\w+\b', str(title).lower())
        frequencies = [word_frequencies[word] for word in title_words]
        return frequencies

    # Apply the function to the column and create a new column
    new_column_name = f"{column_name}_frequencies"
    df[new_column_name] = df[column_name].apply(replace_with_frequency)

    # Calculate the sum of word frequencies for each row
    df[f'{column_name}_frequency_sum'] = df[new_column_name].apply(lambda x: sum(x))
    

    return df

dataset = replace_with_word_frequencies(dataset, 'title')

In [6]:
import pandas as pd

def add_tag_occurrence_list_column(df, column_name):
    # Split the tags in each row and count their occurrences
    all_tags = '|'.join(df[column_name]).split('|')
    tag_counts = pd.Series(all_tags).value_counts()

    # Function to get the total occurrence of a tag in the entire column
    def get_tag_occurrence(tags):
        tag_list = tags.split('|')
        return [tag_counts.get(tag, 0) for tag in tag_list]

    # Create a new column 'tag_occurrence_list' with the total occurrence of tags in each row
    df['tag_frequency_list'] = df[column_name].apply(get_tag_occurrence)

    # Modify 'tag_occurrence_list' to contain the sum of values in each list
    df['tag_frequency_sum'] = df['tag_frequency_list'].apply(lambda x: sum(x))

    return df

# Call the function with your DataFrame and column name
dataset = add_tag_occurrence_list_column(dataset, 'tags')

In [7]:
print(type(dataset['days_to_trend'][0]))
dataset['days_to_trend'] = dataset['days_to_trend'].dt.days
type(dataset['days_to_trend'][0])

<class 'pandas._libs.tslibs.timedeltas.Timedelta'>


numpy.int64

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score


features = ['likes', 'dislikes', 'comment_count', 'categoryId','title_frequency_sum','tag_frequency_sum','days_to_trend']
target = 'view_count'
X = dataset[features]
y = dataset[target]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42)
X_tune, X_test, y_tune, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
dt_model = DecisionTreeRegressor()

# Fit the model to the training data
dt_model.fit(X_train, y_train)

DecisionTreeRegressor()

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


rf_model = RandomForestRegressor(random_state=42)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=2, n_jobs=-1)
grid_search.fit(X_tune, y_tune)

# Get the best parameters from the grid search
best_params = grid_search.best_params_

# Train the Random Forest model with the best parameters
final_rf_model = RandomForestRegressor(random_state=42, **best_params)
final_rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = final_rf_model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)

print(f'R^2 Score on Test Set: {r2:.3f}')

Fitting 3 folds for each of 81 candidates, totalling 243 fits
R^2 Score on Test Set: 0.927


In [10]:
#without tuning
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Define features (X) and target variable (y)
x = ['likes', 'dislikes', 'comment_count', 'categoryId','title_frequency_sum','tag_frequency_sum','days_to_trend']
y = dataset['view_count']
X = dataset[x]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Further split the training set into train and tune sets
X_train, X_tune, y_train, y_tune = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# Initialize XGBoost regressor
xgb_model = XGBRegressor()

# Fit the model to the training data
xgb_model.fit(X_train, y_train)

# Make predictions on the tune set
y_tune_pred = xgb_model.predict(X_tune)

# Evaluate the model on the tune set
r2_tune = r2_score(y_tune, y_tune_pred)

print(f"R^2 Score on Tune Set: {r2_tune}")

# Make predictions on the test set
y_test_pred = xgb_model.predict(X_test)

# Evaluate the model on the test set
r2_test = r2_score(y_test, y_test_pred)

print(f"R^2 Score on Test Set: {r2_test}")

R^2 Score on Tune Set: 0.9158958895836634
R^2 Score on Test Set: 0.920137602392218


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Define features (X) and target variable (y)
x = ['likes', 'dislikes', 'comment_count', 'categoryId','title_frequency_sum','tag_frequency_sum','days_to_trend']

y = dataset['view_count']
X = dataset[x]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Further split the training set into train and tune sets
X_train, X_tune, y_train, y_tune = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# Initialize XGBoost regressor
xgb_model = XGBRegressor()

# Define hyperparameter grid for tuning
param_grid = {

    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],

}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_

# Use the best parameters to create a tuned XGBoost model
tuned_xgb_model = XGBRegressor(**best_params)

# Fit the tuned model to the training data
tuned_xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_test_pred = tuned_xgb_model.predict(X_test)

# Evaluate the model on the test set
r2_test = r2_score(y_test, y_test_pred)

print(f"Best Parameters: {best_params}")
print(f"R^2 Score on Test Set: {r2_test}")


Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.9}
R^2 Score on Test Set: 0.9304894477646157
